In [ ]:
import tensorflow as tf
import h5py
from tensorflow.python.keras.utils.np_utils import to_categorical


bOur goal is to create a dataset consisting of traces and attack points for model learning, and we will utilize plaintext and keys for this. We will use the stored information from data and key sets to do this. traces will function as X and attack points will be Y. In this notebook we are testing the data. The test dataset will be gathered on the same chipwhisperer, but with a new key each time?

Right now, for this version i only gather 65636 traces, but i also collect the sub_bytes_out from the trace, this seems to take up a lot more time then previous, when we only gathered text, key and trace.

In [ ]:
#Loading 256*500 for testin, usefull to not use up all my memory
# Load the h5py file
file_path = "C:\\Users\\Kaspar\\ChipWhisperer5_64\\cw\\home\\portable\\chipwhisperer\\jupyter\\courses\\sca101\\TINYAES_test_set.hdf5"
with h5py.File(file_path, "r") as h5_file:
    plaintext_set = h5_file['data'][:65536]
    trace_set = h5_file['trace'][:65536]
    key_set = h5_file['key'][:65536]
    #sub_bytes_out_set = h5_file['sub_bytes_out'][:65536]


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.plot(trace_set[0][:100])

We create a dataset consisting og plaintext and keys so that we can pre-compute sub_bytes_in, which is one of the attack points. This is done by XOR on key and plaintext

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((plaintext_set_set, key_set))


We also have to pre compute our attack points, as we use CW firmware we do not have the SBOX available at the moment, and will therefore rely more on sub_bytes_in


In [ ]:
##Kan lage en loop over denne for å beregne alle attack points, men det får bli senere


# Define dataset variables for attack data
attack_byte = []

# Calculate attack points
for key, data in dataset:
    # Compute relevant attack point, assuming sub_byte_in as an example
    # Here, we're aiming to simulate an attack point based on specific indexing logic
    # Assuming sub_byte_index refers to the index of the targeted byte
    sub_byte_index = 0  # Example: targeting the first byte
    sub_byte_in = data[sub_byte_index] ^ key[sub_byte_index]

    # Store computed attack point
    attack_byte.append(sub_byte_in)

#sørger for at punktene er på riktig format
#attack_byte = to_categorical(attack_points, 256)
# Convert attack points to a TensorFlow tensor for compatibility with training
attack_byte = tf.convert_to_tensor(attack_byte, dtype=tf.uint8)



Making sure that the matrix is of the form byte_id, example_id

In [ ]:
attack_byte.shape

In [ ]:
key_set.shape

In [ ]:
trace_set.shape

In [ ]:
data_set.shape

As we can see, the data is stored the "wrong" way, we want byteid, example so we must transpose it in the process

In [ ]:
print(attack_byte[0])

Adding attack points to the dataset, in this case this is for index 0

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data_set, trace_set, key_set, attack_byte))

At the moment i can fit everything in memory, when using the whole dataset the shuffle buffer will be 128 000, because of us using 500 traces per key, this should be enough. Batch size might vary

Feature scaling on the power trace, so all values will be between -1 and 1(scaaml advice for making models converge

In [ ]:
trace_set = tf.keras.layers.Rescaling(1./127.5, offset=-1)(trace_set)

In [ ]:
print(trace_set[0])

In [ ]:
trace_set = tf.expand_dims(trace_set, axis=-1)

Making sure that the values are scaled properly, and adding another dimension to the trace. This makes sure that it is compatible with Conv1D layers

In [ ]:
print(trace_set[10])

Shard seems to have a 3x16 matrix with keys, subin and subout, and then the bytes
it loops through and gets the attack byte of the specified point from all the shards as y_shard and something else for x_shard. So i must save all the shards, and then get them. seems like the dataset only ends up with 256 points though?

500 traces per key, 500*16 attack bytes and 3 attack points. but when loading a shard it fetches shard[attack_point][attack_byte], and then y =y[:num_traces:per_shard] which will be 500, so it gets them all for this byte?